In [3]:
suppressPackageStartupMessages({
    library(keras)
    library(tidyverse)
    library(quantmod)
    library(timetk)
    library(lubridate)
    library(tidymodels)
    library(kmscv)
})

Warning message:
"package 'quantmod' was built under R version 4.2.1"
Warning message:
"package 'kmscv' was built under R version 4.2.1"


In [4]:
set.seed(1)

spc <- boost_tree(trees = tune(),min_n = tune(),tree_depth = tune(),learn_rate = tune()) %>%
set_mode('regression') %>%
set_engine('xgboost')

rsmpl <- vfold_cv(iris,v = 5)

rcp <- Sepal.Width ~ . - Species

metrik <- yardstick::metric_set(mae,mape)

In [14]:
tn <- tune::tune_bayes(spc,resamples = rsmpl2,iter = 15,metrics = metrik,preprocessor = rcp,control = control_bayes(verbose = T))



❯  Generating a set of 5 initial parameter results

✓ Initialization complete



Optimizing mae using the expected improvement



── Iteration 1 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────



i Current best:		mae=0.214 (@iter 0)

i Gaussian process model

! The Gaussian process model is being fit using 4 features but only has 5
  data points to do so. This may cause errors or a poor model fit.

✓ Gaussian process model

i Generating 5000 candidates

i Predicted candidates

i trees=1952, min_n=23, tree_depth=2, learn_rate=0.0149

i Estimating performance

✓ Estimating performance

♥ Newest results:	mae=0.209 (+/-0.00519)



── Iteration 2 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────



i Current best:		mae=0.209 (@iter 1)

i Gaussian process model

✓ Gaussian process model

i Generating 5000 candidates

i Predict

In [13]:
rsmpl %>% select(splits) %>% pull %>% lapply(complement) %>% unlist %>% length

[1] 150

In [12]:
rsmpl2 %>% select(splits) %>% pull %>% lapply(complement) %>% unlist %>% length

[1] 150

In [ ]:
rsmpl$split

In [3]:
get_scaling_factors <- function(data){
  out <- c(mean = mean(data), sd = sd(data))
  return(out)
}

normalize_data <- function(data, scaling_factors, reverse = FALSE) {
  
  if (reverse) temp <- (data * scaling_factors[2]) + scaling_factors[1]
  else temp <- (data - scaling_factors[1]) / scaling_factors[2]
  
  out <- temp %>% as.matrix()
  return(out)
}

In [10]:
kerasize_data <- function(data, x = TRUE, lag = 12, pred = 12) {
  
  if (x) {
    temp <- sapply(
      1:(length(data) - lag - pred + 1)
      ,function(x) data[x:(x + lag - 1), 1]
    ) %>% t()
    
    out <- array(
      temp %>% unlist() %>% as.numeric()
      ,dim = c(nrow(temp), lag, 1)
    )
    
  }  else {
    
    temp <- sapply(
      (1 + lag):(length(data) - pred + 1)
      ,function(x) data[x:(x + lag - 1), 1]
    ) %>% t()
    
    out <- array(
      temp %>% unlist() %>% as.numeric()
      ,dim = c(nrow(temp), pred, 1)
    ) 
  }
  return(out) 
}

kerasize_pred_input <- function(data, lag = 12, pred = 12){
  temp <- data[(length(data) - pred + 1):length(data)]
  temp <- normalize_data(temp, get_scaling_factors(data))
  out <- array(temp, c(1, lag, 1))
  return(out)
}

In [11]:
lstm_build_model <- function(x, y, units = 50, batch = 1, epochs = 20, rate = 0.5, seed = 2137){
  
  lag = dim(x)[2]
  
  lstm_model <- keras_model_sequential()

  lstm_model %>%
    layer_lstm(units = units
               ,batch_input_shape = c(batch, lag, 1)
               ,return_sequences = TRUE
               ,stateful = TRUE) %>%
    layer_dropout(rate = rate) %>%
    layer_lstm(units = units
               ,return_sequences = TRUE
               ,stateful = TRUE) %>%
    layer_dropout(rate = rate) %>%
    time_distributed(layer_dense(units = 1))

  lstm_model %>%
    compile(loss = 'mae'
            ,optimizer = 'adam'
            ,metrics = 'accuracy')

  tensorflow::set_random_seed(seed)
  lstm_model %>% fit(
    x = x
    ,y = y
    ,batch_size = batch
    ,epochs = epochs
    ,verbose = 0
    ,shuffle = FALSE)
  
  out <- list(
    model = lstm_model
    ,x = x
    ,batch = batch
    ,lag = lag
    ,pred = dim(y)[2]
  )
  return(out)

}

In [12]:
lstm_forecast <- function(x_test, model, scaling_factors){
  
  batch <- model$batch
  
  temp <- model$model %>%
    predict(x_test, batch_size = batch) %>% 
    .[, , 1] %>%
    normalize_data(scaling_factors = scaling_factors, reverse = TRUE)
  
  out <- list(
    forecast = temp
    ,scaling_factors = scaling_factors
  )
  
  return(out)
  
}

In [13]:
forecast_transform <- function(data, model, forecast){
  
  lag <- model$lag
  
  freq <- periodicity(data)$scale
  frequency <- case_when(
    freq == 'weekly' ~ 52
    ,freq == 'monthly' ~ 12
   , freq == 'quarterly' ~ 4
    ,freq == 'yearly' ~ 1
  )
  
  dates <- index(data)
  date_start <- c(year(min(dates)), month(min(dates)), day(min(dates)))
  date_end <- c(year(max(dates)), month(max(dates)), day(max(dates)))
  
  date_start_shifted <- case_when(
    freq == 'weekly' ~ min(dates) %m+% weeks(lag)
    ,freq == 'monthly' ~ min(dates) %m+% months(lag)
    ,freq == 'quarterly' ~ min(dates) %m+% months(3*lag)
    ,freq == 'yearly' ~ min(dates) %m+% years(lag)
    )
  date_start_shifted <- c(year(date_start_shifted), month(date_start_shifted), day(date_start_shifted))
  
  date_end_shifted <- case_when(
    freq == 'weekly' ~ max(dates) %m+% weeks(1)
    ,freq == 'monthly' ~ max(dates) %m+% months(1)
    ,freq == 'quarterly' ~ max(dates) %m+% months(3)
    ,freq == 'yearly' ~ max(dates) %m+% years(1)
    )
  date_end_shifted <- c(year(date_end_shifted), month(date_end_shifted), day(date_end_shifted))

  fitted <- predict(model$model, model$x, batch_size = model$batch) %>% .[, , 1]
  
  if (dim(fitted)[2] > 1) {
    fitted <- c(fitted[, 1], fitted[dim(fitted)[1], 2:dim(fitted)[2]])
  } else {
    fitted <- fitted[, 1]
  }
  
  fitted <- normalize_data(fitted, forecast$scaling_factors, reverse = TRUE)
  fitted <- ts(fitted, start = date_start_shifted, deltat = 1/frequency)
  
  lstm_forecast <- ts(forecast$forecast, start = date_end_shifted, deltat = 1/frequency)
  
  data_trimmed <- data[(model$lag+1):nrow(data)] %>% ts(start = date_start_shifted
                                                        ,deltat = 1/frequency)
  
  
  out <- list(
    model = NULL
    ,method = 'LSTM'
    ,mean = lstm_forecast
    ,x = data_trimmed
    ,fitted = fitted
    ,residuals = as.numeric(data_trimmed) - as.numeric(fitted)
  )
  
  class(out) <- 'forecast'
  
  return(out)
}

In [19]:
data <- getSymbols('MRTSSM4453USS', src = 'FRED', auto.assign = FALSE) 

In [27]:
scaling_factors <- get_scaling_factors(data$MRTSSM4453USS)
data_normalized <- normalize_data(data$MRTSSM4453USS, scaling_factors)

x_data <- kerasize_data(data_normalized,  x = TRUE)
y_data <- kerasize_data(data_normalized, x = FALSE)
x_test <- kerasize_pred_input(data_normalized)

model <- lstm_build_model(x_data, y_data)
prediction <- lstm_forecast(x_test, model, scaling_factors)

final_results <- forecast_transform(data, model, prediction)

In [38]:
data

           MRTSSM4453USS
1992-01-01          1713
1992-02-01          1763
1992-03-01          1753
1992-04-01          1784
1992-05-01          1783
1992-06-01          1782
1992-07-01          1790
1992-08-01          1845
1992-09-01          1836
1992-10-01          1854
1992-11-01          1853
1992-12-01          1856
1993-01-01          1840
1993-02-01          1836
1993-03-01          1814
1993-04-01          1792
1993-05-01          1791
1993-06-01          1796
1993-07-01          1799
1993-08-01          1786
1993-09-01          1767
1993-10-01          1768
1993-11-01          1780
1993-12-01          1773
1994-01-01          1796
1994-02-01          1809
1994-03-01          1825
1994-04-01          1832
1994-05-01          1858
1994-06-01          1844
1994-07-01          1862
1994-08-01          1840
1994-09-01          1864
1994-10-01          1852
1994-11-01          1839
1994-12-01          1841
1995-01-01          1832
1995-02-01          1802
1995-03-01          1810
